<a href="https://colab.research.google.com/github/Athul-nambiar/Fine-Tuning-/blob/main/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate bitsandbytes


In [2]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "aboonaji/llama2finetune-v2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Corrected BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Use 8-bit quantization
    llm_int8_threshold=6.0,
    llm_int8_enable_fp32_cpu_offload=True  # ✅ Correct setting for CPU offloading
)

# Corrected Model Loading with Custom Device Map
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"  # ✅ Auto-distributes model across CPU/GPU
)

print("✅ Model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded successfully!


In [3]:
import torch

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"


# Define your prompt
prompt = "can you give me about stone man syndrome?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=250,  # Adjust as needed
        temperature=0.5,  # Controls randomness (lower = more focused)
        top_p=1,  # Nucleus sampling (higher = more diverse output)
        repetition_penalty=1.1,  # Reduce repetitive text
    )

# Decode and print result
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


can you give me about stone man syndrome?

Stone Man Syndrome, also known as Fibrodysplasia Ossificans Progressiva (FOP), is a rare genetic disorder that affects the body's musculoskeletal system. It is characterized by the progressive and unusual growth of bone tissue in soft tissues such as muscles, tendons, and ligaments. This can lead to the formation of bony outgrowths or "bony knots" that can cause pain, stiffness, and limited mobility in the affected areas.

The symptoms of Stone Man Syndrome typically become apparent during childhood or adolescence and can worsen over time. The disorder is caused by a mutation in the ACVR1 gene, which codes for a protein called activin receptor type 1. This protein plays a role in the development and maintenance of bone tissue.

There is currently no cure for Stone Man Syndrome, but there are several treatments available to help manage the symptoms. These may include:

1. Pain management: Medications such as nonsteroidal anti-infl


In [12]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("aboonaji/wiki_medical_terms_llam2_format")



In [14]:
# Check available splits
print(ds.keys())

# Access the training split
train_dataset = ds["train"]


dict_keys(['train'])


In [22]:
from sklearn.model_selection import train_test_split

# Access the 'train' split from the dataset
train_data = ds["train"]

# Convert to pandas DataFrame for easier manipulation
train_df = train_data.to_pandas()

# Split into train and test sets (e.g., 80% train, 20% test)
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Convert back to datasets.Dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Check the splits
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Train dataset size: 5488
Test dataset size: 1373


In [34]:
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset

# Load the tokenizer (choose the one based on your model)
tokenizer = AutoTokenizer.from_pretrained('aboonaji/llama2finetune-v2')

# Define the preprocess function for tokenization
def preprocess_function(examples):
    # Assuming the 'text' field contains the raw input text
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)

# Apply preprocessing to the training dataset
train_tokenized_ds = train_dataset.map(preprocess_function, batched=True)

# Apply preprocessing to the test dataset
test_tokenized_ds = test_dataset.map(preprocess_function, batched=True)

# Check a sample from the tokenized training dataset to verify
print(train_tokenized_ds[0])  # Prints the first tokenized example

# Optionally, check a sample from the test dataset
print(test_tokenized_ds[0])  # Prints the first tokenized example from test


Map:   0%|          | 0/5488 [00:00<?, ? examples/s]

Map:   0%|          | 0/1373 [00:00<?, ? examples/s]

{'text': "<s> [INST] <<SYS>> You are a helpful, respectful, and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> What is Perinephritis and explain in detail? [/INST]  </s>", '__index_level_0__': 4684, 'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

{'text': '<s> [INST] <<SYS>> You are a helpful, respectful, and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information. <</SYS>> What is Paracetamol poisoning and explain in detail? [/INST] Paracetamol poisoning, also known as acetaminophen poisoning, is caused by excessive use of the medication paracetamol (acetaminophen). Most people have few or non-specific symptoms in the first 24 hours following overdose. These include feeling tired, abdominal pain, or nausea. This is typically followed by a couple of days without any symptoms, after which yellowish skin, blood clotting 

❌ LoRA is NOT applied. You need to wrap the model with LoRA first.
